In [12]:
from pandas import DataFrame
import pandas as pd
import numpy as np

In [13]:
from jsonapi_client import Session, Filter
import pycurl

API_BASE = 'https://www.ebi.ac.uk/metagenomics/api/latest/'

In [14]:
# 'studies' bisa diganti dengan target semisal 'sample' ; 'MGYS00001939' merupakan accession
with Session(API_BASE) as s:
    std = s.get('studies', 'MGYS00001939').resource

In [15]:
#memunculkan study_name
std

<ResourceObject: studies: MGYS00001939 (1999326051968) (1999326051968)>

In [16]:
#melihat jumlah sample pada study di atas
std.samples_count

12

In [17]:
#memunculkan list sample sesuai jumlah
std.samples

[<ResourceObject: samples: ERS1851088 (1999326125696) (1999326125696)>,
 <ResourceObject: samples: ERS1851084 (1999326288584) (1999326288584)>,
 <ResourceObject: samples: ERS1851093 (2001453890528) (2001453890528)>,
 <ResourceObject: samples: ERS1851087 (1999326326008) (1999326326008)>,
 <ResourceObject: samples: ERS1851094 (1999326409000) (1999326409000)>,
 <ResourceObject: samples: ERS1851086 (1999326410008) (1999326410008)>,
 <ResourceObject: samples: ERS1851095 (1999326411016) (1999326411016)>,
 <ResourceObject: samples: ERS1851090 (1999326412024) (1999326412024)>,
 <ResourceObject: samples: ERS1851092 (1999326437672) (1999326437672)>,
 <ResourceObject: samples: ERS1851085 (1999326438680) (1999326438680)>,
 <ResourceObject: samples: ERS1851091 (1999326439688) (1999326439688)>,
 <ResourceObject: samples: ERS1851089 (1999326440696) (1999326440696)>]

In [9]:
sample = std.samples
#melihat informasi pada salah satu sample yang dipilih
x=sample[0]
x.sample_metadata

[{'key': 'investigation type', 'unit': None, 'value': 'metatranscriptome'},
 {'key': 'geographic location (longitude)', 'unit': None, 'value': '5.411394'},
 {'key': 'geographic location (country and/or sea,region)',
  'unit': None,
  'value': 'Belgium'},
 {'key': 'collection date', 'unit': None, 'value': '2015-03-18'},
 {'key': 'environment (biome)', 'unit': None, 'value': 'area of cropland'},
 {'key': 'environment (feature)', 'unit': None, 'value': 'area of cropland'},
 {'key': 'environment (material)', 'unit': None, 'value': 'agricultural soil'},
 {'key': 'environmental package', 'unit': None, 'value': 'soil'},
 {'key': 'sequencing method', 'unit': None, 'value': 'Illumina'},
 {'key': 'elevation', 'unit': None, 'value': '46.0'},
 {'key': 'geographic location (latitude)', 'unit': None, 'value': '50.93487'},
 {'key': 'instrument model', 'unit': None, 'value': 'Illumina HiSeq 4000'}]

In [35]:
blank = []
for x in sample:
    #print(x.sample_metadata)
    df = pd.DataFrame(x.sample_metadata)
    #print(df)
    index = np.where(df['key'].str.contains('location'))[0]
    blank.append(df.iloc[index, :])
    index = np.where(df['key'].str.contains('biome'))[0]
    blank.append(df.iloc[index, :])
    index = np.where(df['key'].str.contains('elevation'))[0]
    blank.append(df.iloc[index, :])
    df_new = pd.concat(blank)
df_new

,key,unit,value
1,geographic location (longitude),None,5.411394
2,"geographic location (country and/or sea,region)",None,Belgium
10,geographic location (latitude),None,50.93487
4,environment (biome),None,area of cropland
9,elevation,None,46.0
1,geographic location (longitude),None,5.411394
2,"geographic location (country and/or sea,region)",None,Belgium
10,geographic location (latitude),None,50.93487
4,environment (biome),None,area of cropland
9,elevation,None,46.0


In [18]:
#memasukkan sample yang dipilih ke bentuk kolom
df = pd.DataFrame(sample[0].sample_metadata)
df

,key,unit,value
0,investigation type,None,metatranscriptome
1,geographic location (longitude),None,5.411394
2,"geographic location (country and/or sea,region)",None,Belgium
3,collection date,None,2015-03-18
4,environment (biome),None,area of cropland
5,environment (feature),None,area of cropland
6,environment (material),None,agricultural soil
7,environmental package,None,soil
8,sequencing method,None,Illumina
9,elevation,None,46.0


In [19]:
blank = []

In [20]:
#membuat index baru dari list 'key' yang dipilih
index = np.where(df['key'].str.contains('location'))[0]
blank.append(df.iloc[index, :])
index = np.where(df['key'].str.contains('biome'))[0]
blank.append(df.iloc[index, :])
index = np.where(df['key'].str.contains('elevation'))[0]
blank.append(df.iloc[index, :])

In [21]:
#'key' yang dilih dijadikan data frame baru
df_new = pd.concat(blank)
df_new

,key,unit,value
1,geographic location (longitude),None,5.411394
2,"geographic location (country and/or sea,region)",None,Belgium
10,geographic location (latitude),None,50.93487
4,environment (biome),None,area of cropland
9,elevation,None,46.0


In [8]:
filename_sample = ''
    

In [27]:
#membuat loop accession di dalam bentuk list
for i in sample:
    #print('sample: '+str(i.accession))
    for x in i.runs:
        #print('runs: '+str(x.accession))
        for y in x.analyses:
            #print('analyses: '+str(y.accession))
            for z in y.downloads:
                if 'json.biom' in z.url:
                    print(z.url)

https://www.ebi.ac.uk/metagenomics/api/v1/analyses/MGYA00135103/file/ERR2061845_MERGED_FASTQ_otu_table_json.biom
https://www.ebi.ac.uk/metagenomics/api/v1/analyses/MGYA00135102/file/ERR2061841_MERGED_FASTQ_otu_table_json.biom
https://www.ebi.ac.uk/metagenomics/api/v1/analyses/MGYA00135099/file/ERR2061850_MERGED_FASTQ_otu_table_json.biom
https://www.ebi.ac.uk/metagenomics/api/v1/analyses/MGYA00135098/file/ERR2061844_MERGED_FASTQ_otu_table_json.biom
https://www.ebi.ac.uk/metagenomics/api/v1/analyses/MGYA00135093/file/ERR2061851_MERGED_FASTQ_otu_table_json.biom
https://www.ebi.ac.uk/metagenomics/api/v1/analyses/MGYA00135104/file/ERR2061843_MERGED_FASTQ_otu_table_json.biom
https://www.ebi.ac.uk/metagenomics/api/v1/analyses/MGYA00135094/file/ERR2061852_MERGED_FASTQ_otu_table_json.biom
https://www.ebi.ac.uk/metagenomics/api/v1/analyses/MGYA00135095/file/ERR2061847_MERGED_FASTQ_otu_table_json.biom
https://www.ebi.ac.uk/metagenomics/api/v1/analyses/MGYA00135096/file/ERR2061849_MERGED_FASTQ_otu

In [30]:
with open('study/'+i.accession+'.biom', 'wb') as f:
            c = pycurl.Curl()
            c.setopt(c.URL, z.url)
            c.setopt(c.WRITEDATA, f)
            c.perform()
            c.close()

FileNotFoundError: [Errno 2] No such file or directory: 'study/ERS1851089.biom'